In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.outliers_influence as oi
from statsmodels.tools.tools import pinv_extended
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn import metrics
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Laptop_Data_Cleaned = pd.read_csv('/content/drive/MyDrive/Personal Files/Regression Practice/Laptop_Data_Cleaned.csv')
Laptop_Data_Cleaned.head()

,Weight (kg),Price,Company_Asus,Company_Dell,Company_HP,Company_Lenovo,Company_MSI,Company_Other,Company_Toshiba,OpSys_Linux,...,ScreenResolution_1600x900,ScreenResolution_1920x1080,ScreenResolution_2560x1440,ScreenResolution_3200x1800,ScreenResolution_3840x2160,ScreenDisplayType_Full HD,ScreenDisplayType_None,ScreenDisplayType_Quad HD+,ScreenIPSPanel_Yes,ScreenTouchscreen_Yes
0,1.86,30636.000,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,2.10,21312.000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1.30,79653.600,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,1.60,41025.600,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
4,1.86,20986.992,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
# Set the target variable, split the data, and normalize it
# Parts of this cell were gotten from freecodecamp site

X = Laptop_Data_Cleaned.drop('Price', axis=1)
y = Laptop_Data_Cleaned['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_normalized = scaler.transform(X_train)
X_train_normalized = sm.add_constant(X_train_normalized)

scaler.fit(X_test)
X_test_normalized = scaler.transform(X_test)
X_test_normalized = sm.add_constant(X_test_normalized)

In [ ]:
# Perform OLS for comparison with other models

model = sm.OLS(y_train, X_train_normalized).fit()
print(model.summary(xname = ['const'] + X_train.columns.tolist()))

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     58.99
Date:                Tue, 13 May 2025   Prob (F-statistic):          2.76e-259
Time:                        21:09:52   Log-Likelihood:                -10450.
No. Observations:                 935   AIC:                         2.102e+04
Df Residuals:                     877   BIC:                         2.130e+04
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [ ]:
# Fit Elastic Net Model to Reduce Multicollinearity
# Function gotten from https://stackoverflow.com/questions/68222193/calculate-adjusted-r2-using-gridsearchcv

def adj_r2(estimator, X_train_normalized, y_train):
  n, p = X_train_normalized.shape
  y_pred = estimator.predict(X_train_normalized)
  return 1 - ((1 - metrics.r2_score(y_train, y_pred)) * (n - 1))/(n - p - 1)

pipe = make_pipeline(MinMaxScaler(), ElasticNet())

param_grid = {'elasticnet__alpha': [1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0], 'elasticnet__l1_ratio': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
cv = RepeatedKFold(n_splits = 10, n_repeats = 5, random_state = 1)

grid_search = GridSearchCV(pipe, param_grid, cv=cv, scoring=adj_r2)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.367e+11, tolerance: 1.264e+08 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.353e+11, tolerance: 1.253e+08 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

GridSearchCV(cv=RepeatedKFold(n_repeats=5, n_splits=10, random_state=1),
             estimator=Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                                       ('elasticnet', ElasticNet())]),
             param_grid={'elasticnet__alpha': [0.001, 0.01, 0.1, 0.0, 1.0,
                                               10.0],
                         'elasticnet__l1_ratio': [0, 0.1, 0.2, 0.3, 0.4, 0.5,
                                                  0.6, 0.7, 0.8, 0.9, 1.0]},
             scoring=<function adj_r2 at 0x7cd9c813cc20>)

In [ ]:
best_alpha, best_l1_ratio = grid_search.best_params_.values()
print(f"Best Aplha: {best_alpha}, Best L1 Ratio: {best_l1_ratio}")

model_2 = sm.OLS(y_train, X_train_normalized)
model_2_regularized = model_2.fit_regularized(method = 'elastic_net', alpha = best_alpha, L1_wt = best_l1_ratio)

pinv_wexog,_ = pinv_extended(model_2.wexog)
model_2_regularized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))

summary = sm.regression.linear_model.OLSResults(model_2, model_2_regularized.params, model_2_regularized_cov_params)
print(summary.summary(xname = ['const'] + X_train.columns.tolist()))

Best Aplha: 0.01, Best L1 Ratio: 0.9
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     57.17
Date:                Tue, 13 May 2025   Prob (F-statistic):          1.21e-254
Time:                        21:11:43   Log-Likelihood:                -10462.
No. Observations:                 935   AIC:                         2.104e+04
Df Residuals:                     877   BIC:                         2.132e+04
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

/usr/local/lib/python3.11/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(eigvals[0]/eigvals[-1])


In [ ]:
# Check for potential endogenous variables

for i in X_train.columns:
  correlation_coefficient = np.corrcoef(X_train[i], model.resid)[0, 1]
  if abs(correlation_coefficient) > 0.7:
    print(i)

In [ ]:
# Reduce multicollinearity
# Remove parameters with high VIF

def create_vif_df():
  vif_df = pd.DataFrame()
  vif_df['Variable'] = X.columns
  vif_df['VIF'] = [oi.variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
  return vif_df

vif_df = create_vif_df()

while max(vif_df['VIF']) > 10:
  dropped_value = vif_df.loc[vif_df['VIF'].idxmax(), 'Variable']
  X = X.drop(dropped_value, axis=1)

  vif_df = create_vif_df()

vif_df.sort_values('VIF', ascending=False)

/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                          Variable        VIF
0                      Weight (kg)   5.155868
1                     Company_Asus   2.614618
2                     Company_Dell   3.381038
3                       Company_HP   3.442444
4                   Company_Lenovo   3.560634
5                      Company_MSI   2.114679
6                    Company_Other   1.564541
7                  Company_Toshiba   1.671922
8                      OpSys_Linux   5.784401
9                      OpSys_No OS   6.313291
10                OpSys_Windows 10  13.188358
11                 OpSys_Windows 7   4.686308
12                 TypeName_Gaming   9.929188
13                TypeName_Netbook   2.602508
14               TypeName_Notebook  10.412741
15              TypeName_Ultrabook   4.464085
16            TypeName_Workstation   2.181069
17                     Inches_12.5   3.922089
18                     Inches_13.3  13.295701
19                     Inches_14.0  15.614125
20                     Inches_15.6

In [ ]:
# Show remaining correlated parameters
# gotten from youtube vid

corr_matrix = X.corr()
for i in range(len(corr_matrix.columns)):
  for j in range(i):
    if abs(corr_matrix.iloc[i, j]) > 0.7:
      print(f'{corr_matrix.columns[i]} and {corr_matrix.columns[j]}')

In [ ]:
# Recombine X and y and save Data

Laptop_Data_Reduced = pd.concat([X, y], axis=1)
Laptop_Data_Reduced.to_csv('/content/drive/MyDrive/Personal Files/Regression Practice/Laptop_Data_Reduced.csv', index=False)